In [1]:
import folium as fol
import pandas as pd
from folium.plugins import MarkerCluster

In [2]:
#Instantiate a map object with the Florence coordinates
m = fol.Map(location=[43.769871, 11.255576], zoom_start=13,tiles='cartodb positron')

In [3]:
#read the excel
df = pd.read_excel('florence_venues5.xlsx', sheet_name='Foglio1')

In [4]:
# Example function to insert decimal point since coordinates
# are stored as integers in the Excel file
# (e.g. 4326 instead of 43.26)
def insert_decimal_point(s):
    return float(s[:2] + '.' + s[2:])

# Read the data


# Apply the conversion
#df['latitude'] = df['latitude'].astype(str).apply(insert_decimal_point)
#df['longitude'] = df['longitude'].astype(str).apply(insert_decimal_point)


In [5]:
def remove_seconds(s):
    return s[:-3]

df['wday_closing'] = df['wday_closing'].astype(str).apply(remove_seconds)
df['wday_opening'] = df['wday_opening'].astype(str).apply(remove_seconds)
df['wkd_closing'] = df['wkd_closing'].astype(str).apply(remove_seconds)
df['wkd_opening'] = df['wkd_opening'].astype(str).apply(remove_seconds)


In [6]:
df.columns

Index(['id', 'venue_name', 'venue_type', 'main_bus', 'venue_add_one',
       'venue_add_two', 'venue_pcode', 'venue_web', 'venue_phone',
       'venue_email', 'venue_fb', 'venue_tw', 'org_own', 'mger', 'mger_email',
       'mger_phone', 'online_contact_form', 'follow_up', 'venue_city',
       'venue_country', 'notes', 'std_sign_off', 'res_sign_off', 's_kick_id',
       'latitude', 'longitude', 'constituency', 'wday_opening', 'wday_closing',
       'wkd_opening', 'wkd_closing', 'show_opening', 'show_closing',
       'variations', 'promoters', 'venue_capacity', 'outdoor_space',
       'type_outdoor_space', 'Instagram', 'PRS Type', 'PRS Venue cat',
       'Year Venue Opened', 'Year Venue Closed', 'Notes', 'Unnamed: 44'],
      dtype='object')

In [7]:
df['venue_type'].unique()

array(['Social', 'Smallvenue', 'Church', 'Rest', 'Outgreen', 'Arena',
       'Stadium', 'hotelres', 'arts', 'Bar, pub with music', 'smallclub',
       'largeclub', 'medvenue', 'concert', 'social', 'rest', 'other',
       'theatre', 'Concert', 'outurban', 'outgreen', 'multivenuefest',
       'largevenue', 'arena', 'student', 'hotres', 'church',
       'bar, pub with music'], dtype=object)

In [8]:
folium_fa_icons = {
    'Social': { 'icon': 'users', 'color': 'blue'},
    'Smallvenue': { 'icon': 'building', 'color': 'green'},
    'Bar, pub with music': { 'icon': 'music', 'color': 'purple'},
    'Church': { 'icon': 'church', 'color': 'darkpurple'},
    'Rest': { 'icon': 'bed', 'color': 'cadetblue'},
    'Outgreen': { 'icon': 'tree', 'color': 'darkgreen'},
    'Arena': { 'icon': 'bullhorn', 'color': 'orange'},
    'Stadium': { 'icon': 'flag-checkered', 'color': 'red'},
    'hotelres': { 'icon': 'hotel', 'color': 'beige'},
    'arts': { 'icon': 'paint-brush', 'color': 'pink'},
    'smallclub': { 'icon': 'glass-martini', 'color': 'lightblue'},
    'largeclub': { 'icon': 'music', 'color': 'darkred'},
    'medvenue': { 'icon': 'building', 'color': 'gray'},
    'concert': { 'icon': 'microphone', 'color': 'black'}
}


In [9]:
folium_fa_icons.update({
    'theatre': { 'icon': 'theater-masks', 'color': 'lightred'},
    'Concert': { 'icon': 'microphone-alt', 'color': 'darkblue'},
    'outurban': { 'icon': 'city', 'color': 'cadetblue'},
    'outgreen': { 'icon': 'leaf', 'color': 'green'},
    'multivenuefest': { 'icon': 'calendar-alt', 'color': 'orange'},
    'largevenue': { 'icon': 'landmark', 'color': 'purple'},
    'arena': { 'icon': 'dove', 'color': 'lightgreen'},
    'student': { 'icon': 'graduation-cap', 'color': 'blue'},
    'other': { 'icon': 'question', 'color': 'grey'},
    'social': { 'icon': 'user-friends', 'color': 'lightblue'},
    'rest': { 'icon': 'utensils', 'color': 'olive'}
})


In [11]:
def check_missing(value, default="N/A"):
    """Return the value or a default string if the value is missing."""
    return value if pd.notna(value) else default
    
def get_icon(venue_type):
    default_icon = {'icon': 'circle', 'color': 'gray'}  # Default vanilla icon
    return folium_fa_icons.get(venue_type, default_icon)


popup_template = """
<h4>{venue_name}</h4>
{address}
{postcode}
{links}
{phone}
{venue_type}
{capacity}
<b>Opening times:</b><br>
{weekdays}
{weekends}
"""
marker_cluster = MarkerCluster().add_to(m)
for index, row in df.iterrows():
    popup_text = popup_template.format(
        venue_name=check_missing(row['venue_name']),
        address=f"<b>Address:</b> {check_missing(row['venue_add_one'])} {check_missing(row['venue_add_two'])}<br>",
        postcode=f"<b>Postcode:</b> {check_missing(row['venue_pcode'])}<br>",
        links=f"<b>Links:</b> <a href='{check_missing(row['venue_web'])}'>Website</a> / <a href='{check_missing(row['venue_fb'])}'>Facebook</a> / <a href='{check_missing(row['Instagram'])}'>Instagram</a><br>",
        phone=f"<b>Phone:</b> {check_missing(row['venue_phone'])}<br>",
        venue_type=f"<b>Venue Type:</b> {check_missing(row['venue_type'])}<br>",
        capacity=f"<b>Capacity:</b> {check_missing(row['venue_capacity'])}<br>",
        weekdays=f"<b>Weekdays Opening:</b> {check_missing(row['wday_opening'])} - {check_missing(row['wday_closing'])}<br>",
        weekends=f"<b>Weekends Opening:</b> {check_missing(row['wkd_opening'])} - {check_missing(row['wkd_closing'])}<br>"
    )
    iframe = fol.IFrame(html=popup_text, width=250, height=250)
    fol.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['venue_name'],
        popup=fol.Popup(iframe, max_width=2650),
        icon=fol.Icon(color=get_icon(row['venue_type'])['color'],prefix='fa', icon=get_icon(row['venue_type'])['icon'])
    ).add_to(marker_cluster)


/var/folders/xd/tzsx5trj73g9l2y1d5slm_3r0000gn/T/ipykernel_2719/2474884695.py:40: UserWarning: color argument of Icon should be one of: {'lightgray', 'darkpurple', 'black', 'lightgreen', 'purple', 'gray', 'white', 'green', 'lightred', 'darkgreen', 'red', 'lightblue', 'blue', 'pink', 'darkblue', 'beige', 'darkred', 'cadetblue', 'orange'}.
  icon=fol.Icon(color=get_icon(row['venue_type'])['color'],prefix='fa', icon=get_icon(row['venue_type'])['icon'])


In [12]:
m

In [ ]:
m.save('florence_venues5.html')